In [ ]:
import pandas as pd
import numpy as np
import pymssql

### Import Data

In [ ]:
df = pd.read_csv('./meme_cleaning.csv')

In [ ]:
df.Created_At.sort_values()

### Getting Hashtags to Play Nice

In [ ]:
#Extract all words that begin with # and turn the results into a dataframe
temp = df['Tweet'].str.lower().str.extractall(r"(#\w+)")
temp.columns = ['unnamed']
# Convert the multiple hashtag values into a list
temp = temp.groupby(level = 0)['unnamed'].apply(list)
# Save the result as a feature in the original dataset
df['hashtags'] = temp

In [ ]:
# search through our new hashtag feature and find unique hashtags; place them into the dictionary in the form {hashtag: number of instances}
uniques_count = {}
for value in df['hashtags']:
    for word in value:
        # If the hashtag is already in the dictionary, increment the value by 1
        if word in uniques_count:
            uniques_count[word] += 1
        else:
            uniques_count[word] = 1

In [ ]:
pd.Series(uniques_count).sort_values(ascending=False).head()

In [ ]:
hashtags = pd.DataFrame(uniques_count.keys(), columns=['hashs'])
hashtags

In [ ]:
len(uniques_count.keys())

### Topics

In [ ]:
trends = pd.read_csv('./trends.csv')
trends = pd.Series(pd.concat((trends.iloc[:,1], trends.iloc[:,2]), axis = 0).unique())

## Insert into database

In [ ]:
server = 'imt563-winter21.database.windows.net'
database = 'Team1'
username = 'imt563'
password = 'SuperSafePassword1234'

conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()

### hashtag

In [ ]:
# for index, row in hashtags.iterrows():
#     cursor.execute("INSERT INTO dbo.hashtag(hashtag_text) values(%s)" % ("\'" + row[0] + "\'"))
#     print(index, row[0])
# conn.commit()
# conn.close()

### user_profile

In [ ]:
user_profile = df[['Twitter_Handle','No_of_Followers']].drop_duplicates(subset = ['Twitter_Handle'])

In [ ]:
for index, row in user_profile.iterrows():
    cursor.execute("INSERT INTO dbo.user_profile(profile_name, follower_count) values(%s, %d)" % ("\'" + row.Twitter_Handle + "\'",row.No_of_Followers))
    print(index, row.Twitter_Handle, row.No_of_Followers)
conn.commit()
conn.close()

### photo

In [ ]:
photo = df[['Image_URL']]

In [ ]:
# for index, row in photo.iterrows():
#     cursor.execute("INSERT INTO dbo.photo(photo_url) values(%s)" % ("\'" + row[0] + "\'"))
#     print(index, row[0])
# conn.commit()
# conn.close()

### temp - Focus on interaction between profiles and post

In [ ]:
temp = df.iloc[:,2:7]
temp['Created_At'] = pd.to_datetime(temp['Created_At']).dt.date.astype(str)
temp['Tweet'] = temp['Tweet'].str.replace("'",'').str.lower()

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(profile_name, post_date, tweet_text, like_count, retweet_count) values(%s, %s, %s, %d, %d)" % ("\'" + row[0] + "\'", row[2], "\'" + row[1] + "\'", row[3], row[4]))
conn.commit()
conn.close()

### temp - Focus on interaction between post and hashtag

In [ ]:
temp = df[['Tweet','hashtags']]
temp['Tweet'] = temp['Tweet'].str.replace("'",'').str.lower()
temp = temp.explode('hashtags')

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(tweet, hashtag) values(%s, %s)" % ("\'" + row[0] + "\'","\'" + row[1] + "\'"))
conn.commit()
conn.close()

### temp - Focus on interaction between post and images

In [ ]:
temp = df[['Tweet','Image_URL']]
temp['Tweet'] = temp['Tweet'].str.replace("'",'').str.lower()

In [ ]:
for index, row in temp.iterrows():
    cursor.execute("INSERT INTO dbo.temp(tweet, photo_url) values(%s, %s)" % ("\'" + row[0] + "\'","\'" + row[1] + "\'"))
conn.commit()
conn.close()

# EDA

Let's take a quick look at our data:

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum().sum()

There are a little over 1300 observations and 11 features. I found 575 missing values, but they are all concentrated in the 'User_Location' feature, which will not be overly helpful in predicting engagement.

In [ ]:
X = df[['hashtags','Image_URL']]

In [ ]:
y = (df['No_of_Retweets']+1)/(df['No_of_Followers']+1)